In [1]:
%load_ext autoreload
#To update the imported files

In [2]:
%autoreload
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import mean_absolute_error
from utilities import *
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [3]:
#Loading the y-paramter from file, might be changed to y_a etc.
train_a = pd.read_parquet('../Data_and_task/A/train_targets.parquet')
train_b = pd.read_parquet('../Data_and_task/B/train_targets.parquet')
train_c = pd.read_parquet('../Data_and_task/C/train_targets.parquet')

In [4]:
#Loading estimated/forecasted training_weather from file
X_train_estimated_a = pd.read_parquet('../Data_and_task/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../Data_and_task/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../Data_and_task/C/X_train_estimated.parquet')

In [5]:
#Loading observed weather from file
X_train_observed_a = pd.read_parquet('../Data_and_task/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../Data_and_task/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../Data_and_task/C/X_train_observed.parquet')

In [6]:
#Loading estimated/forecasted test_weather from file
X_test_estimated_a = pd.read_parquet('../Data_and_task/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../Data_and_task/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../Data_and_task/C/X_test_estimated.parquet')

In [7]:
#Cleaning data such that only wanted features are included
selected_features = [
    "date_forecast", "clear_sky_rad:W", "diffuse_rad:W", "direct_rad:W",
    "absolute_humidity_2m:gm3", "effective_cloud_cover:p",
    "is_day:idx", "is_in_shadow:idx", "fresh_snow_12h:cm", "fresh_snow_1h:cm",
    "fresh_snow_24h:cm", "fresh_snow_3h:cm", "fresh_snow_6h:cm",
    "snow_depth:cm", "snow_melt_10min:mm", "sun_azimuth:d", 
    "sun_elevation:d", "wind_speed_10m:ms"
]
#Defining y_fetures, will always be the same
y_features = ["time", "pv_measurement"]

X_train_observed_a_clean = X_train_observed_a[selected_features].copy()
X_train_estimated_a_clean = X_train_estimated_a[selected_features].copy()

X_test_estimated_a_clean = X_test_estimated_a[selected_features][::4].drop(columns = ["date_forecast"]).copy()
X_test_estimated_b_clean = X_test_estimated_b[selected_features][::4].drop(columns = ["date_forecast"]).copy()
X_test_estimated_c_clean = X_test_estimated_c[selected_features][::4].drop(columns = ["date_forecast"]).copy()

In [8]:
#Training a Linear regression model on X_observed_a and testing it on X_estimated_a and evaluating it on MAE, PURELY for testing!
#See below for how its done when submitting
X_train_observed_a_resized, y_train_observed_a = resize_trainingdata(X_train_observed_a_clean, train_a, "date_forecast", y_features)
X_train_estimated_a_resized, y_train_estimated_a = resize_trainingdata(X_train_estimated_a_clean, train_a, "date_forecast", y_features)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_observed_a_resized)
X_test_scaled = scaler.fit_transform(X_train_estimated_a_resized)

#Training the model
reg = LinearRegression()
reg.fit(X_train_scaled, y_train_observed_a["pv_measurement"])

# Make predictions
y_pred = reg.predict(X_test_scaled)

#Evaluate mean asbolute error
mae = mean_absolute_error(y_train_estimated_a["pv_measurement"], y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 382.9977125715309


/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623

In [9]:
#Training model, running infernece on testing data and saving it in a csv file ready for submission
X_train_observed_a_resized, y_train_observed_a = resize_trainingdata(X_train_observed_a_clean, train_a, "date_forecast", y_features)
X_train_estimated_a_resized, y_train_estimated_a = resize_trainingdata(X_train_estimated_a_clean, train_a, "date_forecast", y_features)

#Merging observed and estimated data to one big df
X_train_a = pd.concat([X_train_observed_a_resized, X_train_estimated_a_resized], ignore_index=True)
y_train_a = pd.concat([y_train_observed_a, y_train_estimated_a], ignore_index=True)
X_test = pd.concat([X_test_estimated_a_clean, X_test_estimated_b_clean, X_test_estimated_c_clean], ignore_index=True)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_a_scaled = scaler.fit_transform(X_train_a)
X_test_a_scaled=scaler.fit_transform(X_test)

#Training the models
#Model A:
reg_a = LinearRegression()
reg_a.fit(X_train_a, y_train_a["pv_measurement"])

# Make predictions for A,B,C on the model trained on only A. Problems with the B and C datasets which need to be explored
y_pred = reg_a.predict(X_test_a_scaled)

#Saving y_pred in a proper csv file
filename = "CSV/second_submission.csv"
submission(filename, y_pred)

/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623